In [1]:
import smtplib
import ssl
import time
import datetime as dt

In [2]:
def send_emails(emails,message):
    """ This functions runs SMTPlib to send from
    uccweatheralert@outlook.com """
    port = 587  # For starttls
    smtp_server = "smtp.office365.com"
    sender_email = "uccweatheralert@outlook.com"
    password = "Thegogglesdonothing"

    context = ssl.create_default_context()
    with smtplib.SMTP(smtp_server, port) as server:
        #server.ehlo()  # Can be omitted
        server.starttls(context=context)
        #server.ehlo()  # Can be omitted
        server.login(sender_email, password)
        for receiver_email in emails:
            server.sendmail(sender_email, receiver_email, message)

def get_temps(loggerdatafile):
    """ This function gets temperatures from the logger, columns 5 and 7
    for air and ground, respectively.
    Converted to floats"""
    with open(loggerdatafile, "r") as f1:
        last_line = f1.readlines()[-1]
        last_line = last_line.strip('\n')
        last_line = last_line.split(',')
        T_g = float(last_line[6])
        T_a = float(last_line[4])
    return T_g,T_a

In [3]:
##### PARAMETERS

## Temperature
Tcold = 10  # Cold Threshold
Twarm = 1.0  # Warm Threshold

## Logger
loggerdatafile = r'' # Path and name of the minute data as raw string

## Emails
sendto = []

sendtotest = [] ### FOR TESTING

## Cold and Warm email messages
messagecold = """\
Subject: "Weather Alert: Ground temperature below 0C

UCC Weather Alert

A ground temperature of less than 0 C has been detected by the UCC weather station located in the UCC North Mall Campus.
A subsequent email will be sent once the ground temperature has risen above 2 C.

Please note: This is an automatically generated email, please do not reply to this message. 
For queries or more information please contact Andy Ruth at a.ruth@ucc.ie or Mixtli Campos at mcampos@ucc.ie

Location: 51deg 53' 58.1"N 8deg 29' 09.1"W"""

messagewarm = """\
Subject: Weather Alert UPDATE: Ground temperature above 2C

UCC Weather Alert - UPDATE

The UCC weather station, located in the UCC North Mall Campus, has detected that the ground temperature is currently at or above 2 C.

Please note: This is an automatically generated email, please do not reply to this message.
For queries or more information please contact Andy Ruth at a.ruth@ucc.ie or Mixtli Campos at mcampos@ucc.ie

Location: 51deg 53' 58.1"N 8deg 29' 09.1"W"""    
   

In [4]:
x = True
while x:
    try:
        T_ground,T_air = get_temps(loggerdatafile)
        T_g_str = "Current ground temperature is %.2f C" %T_ground
        T_a_str = "Current air temperature is %.2f C" %T_air
        print(T_g_str,'\n'+T_a_str)
        if T_ground < Tcold :
            sendmessagecold = messagecold+"\n"+T_g_str+"\n"+T_a_str
            send_emails(sendtotest,sendmessagecold)
            y = True
            time.sleep(5)
            while y:
                try:
                    T_ground,T_air = get_temps(loggerdatafile)
                    T_g_str = "Current ground temperature is %.2f C" %T_ground
                    T_a_str = "Current air temperature is %.2f C" %T_air
                    print(T_g_str,'\n'+T_a_str)
                    if T_ground > Twarm :
                        sendmessagewarm = messagewarm+"\n"+T_g_str+"\n"+T_a_str
                        send_emails(sendtotest,sendmessagewarm)
                        y = False
                except:
                    print("Cannot open file, retrying soon ... ")
                    time.sleep(5)
    except:
        print("Cannot open file, retrying ... ")
        time.sleep(5)
    x=False

Current ground temperature is 2.23 C 
Current air temperature is 5.17 C
Current ground temperature is 2.23 C 
Current air temperature is 5.19 C


In [ ]:
while True:
    try:
        T_ground,T_air = get_temps(loggerdatafile)
        T_g_str = "Current ground temperature is %.2f C" %T_ground
        T_a_str = "Current air temperature is %.2f C" %T_air
        print(T_g_str,'\n'+T_a_str)
        if T_ground < Tcold :
            sendmessagecold = messagecold+"\n"+T_g_str+"\n"+T_a_str
            #send_emails(sendtotest,sendmessagecold)
            y = True
            while y:
                time.sleep(600)
                try:
                    T_ground,T_air = get_temps(loggerdatafile)
                    T_g_str = "Current ground temperature is %.2f C" %T_ground
                    T_a_str = "Current air temperature is %.2f C" %T_air
                    print(T_g_str,'\n'+T_a_str)
                    if T_ground > Twarm :
                        sendmessagewarm = messagewarm+"\n"+T_g_str+"\n"+T_a_str
                        #send_emails(sendtotest,sendmessagewarm)
                        y = False
                except:
                    print("Cannot open file, retrying ... ")
                    time.sleep(5)
        time.sleep(600)
    except:
        print("Cannot open file, retrying ... ")
        time.sleep(5)